In [ ]:
# nbi:hide_in
# nbi:hide_out

In [2]:
# nbi:hide_in
import threading
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, Layout
import IPython
import numpy as np
import nbinteract as nbi
from math import cos,pi
import time

# import matplotlib.pyplot

# def vibr(t,f):
#     return A*cos(2*pi*f*t)


# label = Layout(width='120px',height='30px')

############################# WIDGETY #############################

f_wid = widgets.FloatSlider(
    value=5.0,
    min=0.0,
    max=10.0,
    step=0.1,
    description='f [Hz]:',
    disabled=False,
    continuous_update=True,
#     orientation='vertical',
    readout=True,
#     readout_format='f'
)

S_wid = widgets.FloatSlider(
    value=0.5,
    min=0.1,
    max=1.0,
    step=0.05,
    description='S [mm2]:',
    disabled=False,
    continuous_update=True,
#     orientation='vertical',
    readout=True,
#     readout_format='f'
)

d_wid = widgets.FloatSlider(
    value=0.3,
    min=0.1,
    max=0.6,
    step=0.01,
    description='d [mm]:',
    disabled=False,
    continuous_update=True,
#     orientation='vertical',
    readout=True,
#     readout_format='f'
)

A_wid = widgets.FloatSlider(
    value=30,
    min=1,
    max=80,
    step=0.5,
    description='A [mm]:',
    disabled=False,
    continuous_update=True,
#     orientation='vertical',
    readout=True,
#     readout_format='f'
)

style = {'description_width': 'initial'}

A_V = widgets.Text(
    value='initializing',
    placeholder='hello',
    description='Amplituda sygnału:',
    disabled=False,
    style=style
)
A_C = widgets.Text(
    value='initializing',
    placeholder='shalom',
    description='Amplituda sygnału:',
    disabled=False,
    style=style
    
)

############################# FUNKCJE #############################

def f_val():
    return f.value

def times(): return np.arange(0,0.8*np.pi,0.001)

def pojemnosc(x,f,eoer,S,d,A):
    Amp = 0.001*A
    dx = Amp*np.cos(2*pi*f*x)
    ret = eoer*S/(d+dx)
#     print(ret)
    return ret

# Vs - podane napięcie roznicowe = 5V
def napiecie(x, f,eoer,S,d,Vs,A):
    Amp = 0.001*A
    dx = Amp*np.cos(2*pi*f*x)
    C1 = eoer*S/(d+dx)
    C2 = eoer*S/(d-dx)
    Vout = (C1-C2)*Vs/(C1+C2)
    return Vout

def y_values(x, f):
    return 3*np.cos(2*pi*f*x)


def amplitudes(A_V,A_C):
    while(1):
        time.sleep(0.2)
        A_V.value = str(pojemnosc(0,0,1.00054,S_wid.value,d = d_wid.value,A = A_wid.value) )
        A_C.value = str(-1*napiecie(0,0,1.00054,S_wid.value,d = d_wid.value,Vs=5,A = A_wid.value))
    
    
thread = threading.Thread(target=amplitudes, args=(A_V,A_C,))

# def amp_V(A_V):
#     time.sleep(0.2)
#     A_V.value = str(pojemnosc(0,0,1.00054,S_wid.value,d = d_wid.value,A = A_wid.value))
    
# threadA_V = threading.Thread(target=amp_V, args=(A_V,))

opts = {
    'title': 'Zmiana pojemności w czasie',
    'xlabel': 'Czas [s]',
    'ylabel': 'Pojemność [mF]',
    'ylim': (0, 5),
#     'xlim': (0, 700),
    'animation_duration': 100 # spr puzniej
}

# wykres pojemności w czasie, zal. od drgań
cap = nbi.line(times, pojemnosc, f= f_wid, eoer=fixed(1.00054), S=S_wid,d=d_wid, A=A_wid, options=opts ) 

opts = {
    'title': 'Napięcie wyjściowe akcelerometru',
    'xlabel': 'Czas [s]',
    'ylabel': 'Napięcie Wyjściowe [V]',
    'ylim': (-5, 5),
#     'xlim': (0, 700),
    'animation_duration': 100 # spr puzniej
}

# nbi.line(time, y_values, f=(0.5,10))
voltage = nbi.line(times, napiecie, f= f_wid, eoer=fixed(1.00054), S=S_wid,d=d_wid, Vs=fixed(5),A=A_wid, options=opts)

htmlText = widgets.HTML(
    value='<h2> Symulator Akcelerometru Pojemnościowego </h2> \
    <a href="file:///C:/Users/Wojtek/Dropbox/StudiaMGR_2/Proj sensorkow/P/inst.html">Instrukcja</a> \
    <br><p> f - czestotliwosc drgań obiektu</p> \
    <p> S - suma powierzchni okładek kondensatora grzebieniowego</p> \
    <p> d - dystans między okładkami kondensatora w stanie ustalonym (brak drgań)</p> \
    <p> A - Amplituda drgań wibrującego obiektu </p>',
)

display(htmlText)



VpA = widgets.VBox([voltage,A_V])
CpA = widgets.VBox([cap,A_C])
# interact(ampl_V,x=2)
# av = interactive(pojemnosc,x=fixed(0),f=fixed(0),eoer=fixed(1.00054),S=S_wid, d = d_wid, A = A_wid)

# print(av.result)

dis = widgets.HBox([CpA,VpA])


display(dis)


thread.start()




HTML(value='<h2> Symulator Akcelerometru Pojemnościowego </h2>     <a href="file:///C:/Users/Wojtek/Dropbox/St…